# CCL 樹脂配方 - 反向設計與優化

給定目標規格，使用網格搜尋和遺傳演算法搜尋最佳配方。

In [ ]:
import sys
sys.path.append('../models')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from predictor import CCLPredictor
from optimizer import CCLOptimizer, format_results

plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

## 1. 載入訓練好的模型

In [ ]:
# 載入模型
predictor = CCLPredictor()
predictor.load('../models/ccl_predictor.pkl')

## 2. 設定目標規格

### 5G 高頻 CCL 規格要求

| 性質 | 目標 | 說明 |
|------|------|------|
| Dk @10GHz | ≤ 3.5 | 低介電常數，提高信號速度 |
| Df @10GHz | ≤ 0.010 | 低介電損耗，減少信號衰減 |
| Peel Strength | ≥ 0.7 N/mm | 足夠的銅箔黏著力 |
| Tg | ≥ 160°C | 耐高溫回焊製程 |
| CTE | ≤ 35 ppm | 低熱膨脹，提高可靠性 |

In [ ]:
# 建立優化器
optimizer = CCLOptimizer(predictor)

# 設定目標規格
target_specs = {
    'Dk_10GHz': {'type': 'max', 'value': 3.5, 'weight': 2.0},
    'Df_10GHz': {'type': 'max', 'value': 0.010, 'weight': 2.0},
    'Peel_Strength_N_mm': {'type': 'min', 'value': 0.7, 'weight': 1.5},
    'Tg_C': {'type': 'min', 'value': 160, 'weight': 1.0},
    'CTE_ppm': {'type': 'max', 'value': 35, 'weight': 1.0}
}

optimizer.set_targets(target_specs)

## 3. 方法 1: 網格搜尋

In [ ]:
# 網格搜尋 (隨機採樣 20000 個配方)
results_grid = optimizer.search(method='grid', n_samples=20000, n_results=10)
print(format_results(results_grid, optimizer.target_specs))

In [ ]:
# 顯示表格
display_cols = ['Hardener_Eq_Ratio', 'Filler_Vol_Pct', 'FR_Wt_Pct', 'Toughener_Wt_Pct',
                'Dk_10GHz', 'Df_10GHz', 'Peel_Strength_N_mm', 'Tg_C', 'CTE_ppm', 'satisfied']
results_grid[display_cols].round(3)

## 4. 方法 2: 遺傳演算法

In [ ]:
# 遺傳演算法
results_ga = optimizer.search(
    method='genetic', 
    population_size=200, 
    generations=50, 
    mutation_rate=0.15,
    n_results=10
)
print(format_results(results_ga, optimizer.target_specs))

## 5. 更嚴格的規格測試

In [ ]:
# 更嚴格的規格 (挑戰極限)
strict_specs = {
    'Dk_10GHz': {'type': 'max', 'value': 3.4, 'weight': 2.0},
    'Df_10GHz': {'type': 'max', 'value': 0.008, 'weight': 2.0},
    'Peel_Strength_N_mm': {'type': 'min', 'value': 0.8, 'weight': 1.5},
    'Tg_C': {'type': 'min', 'value': 165, 'weight': 1.0},
    'CTE_ppm': {'type': 'max', 'value': 30, 'weight': 1.0}
}

optimizer.set_targets(strict_specs)
results_strict = optimizer.search(method='genetic', population_size=200, generations=100, n_results=5)
print(format_results(results_strict, optimizer.target_specs))

## 6. 配方空間視覺化

In [ ]:
# 生成大量配方並視覺化
from optimizer import FormulationBounds
import random

bounds = FormulationBounds()
n_viz = 5000

viz_data = []
for _ in range(n_viz):
    f = {
        'Hardener_Eq_Ratio': random.uniform(0.75, 1.15),
        'Filler_Vol_Pct': random.uniform(15, 55),
        'FR_Wt_Pct': random.uniform(3, 8),
        'Toughener_Wt_Pct': random.uniform(0, 8),
        'Wash_Cycles': random.randint(2, 5),
        'Residual_Cl_ppm': random.uniform(5, 45)
    }
    pred = predictor.predict(f)
    viz_data.append({**f, **pred})

df_viz = pd.DataFrame(viz_data)

In [ ]:
# 視覺化: Dk vs Df，顏色表示是否符合 Peel 規格
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 左圖: Dk vs Df
peel_ok = df_viz['Peel_Strength_N_mm'] >= 0.7
axes[0].scatter(df_viz[~peel_ok]['Dk_10GHz'], df_viz[~peel_ok]['Df_10GHz'], 
                alpha=0.3, c='gray', label='Peel < 0.7', s=10)
axes[0].scatter(df_viz[peel_ok]['Dk_10GHz'], df_viz[peel_ok]['Df_10GHz'], 
                alpha=0.5, c='green', label='Peel ≥ 0.7', s=10)
axes[0].axvline(3.5, color='red', linestyle='--', alpha=0.7)
axes[0].axhline(0.010, color='red', linestyle='--', alpha=0.7)
axes[0].set_xlabel('Dk @10GHz')
axes[0].set_ylabel('Df @10GHz')
axes[0].set_title('Dk vs Df (綠色 = Peel ≥ 0.7)')
axes[0].legend()

# 右圖: CTE vs Tg
all_ok = (df_viz['Dk_10GHz'] <= 3.5) & (df_viz['Df_10GHz'] <= 0.010) & \
         (df_viz['Peel_Strength_N_mm'] >= 0.7) & (df_viz['Tg_C'] >= 160) & \
         (df_viz['CTE_ppm'] <= 35)

axes[1].scatter(df_viz[~all_ok]['CTE_ppm'], df_viz[~all_ok]['Tg_C'], 
                alpha=0.3, c='gray', label='未符合全部規格', s=10)
axes[1].scatter(df_viz[all_ok]['CTE_ppm'], df_viz[all_ok]['Tg_C'], 
                alpha=0.7, c='blue', label='符合全部規格', s=20)
axes[1].axvline(35, color='red', linestyle='--', alpha=0.7)
axes[1].axhline(160, color='red', linestyle='--', alpha=0.7)
axes[1].set_xlabel('CTE (ppm)')
axes[1].set_ylabel('Tg (°C)')
axes[1].set_title('CTE vs Tg (藍色 = 符合全部規格)')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f'符合全部規格的配方: {all_ok.sum()}/{n_viz} ({all_ok.mean()*100:.1f}%)')

## 7. 最佳配方建議

In [ ]:
# 統計符合規格的配方參數範圍
df_ok = df_viz[all_ok]

if len(df_ok) > 0:
    print('符合所有規格的配方參數範圍:')
    print('=' * 50)
    for col in ['Hardener_Eq_Ratio', 'Filler_Vol_Pct', 'FR_Wt_Pct', 'Toughener_Wt_Pct']:
        print(f'{col:25s}: {df_ok[col].min():.2f} ~ {df_ok[col].max():.2f} (建議: {df_ok[col].median():.2f})')
else:
    print('沒有配方能同時符合所有規格')

## 8. 結論與建議

### 最佳配方範圍 (5G 高頻 CCL)

| 參數 | 建議範圍 | 說明 |
|------|----------|------|
| 硬化劑當量比 | 1.05 ~ 1.15 | 高當量比降低 Dk/Df |
| 填料體積% | 38 ~ 45 vol% | 平衡 CTE 和 Peel |
| 阻燃劑重量% | 4 ~ 6 wt% | 標準範圍 |
| 增韌劑重量% | 1 ~ 3 wt% | 適量提升 Peel，不過度犧牲 Tg |

### 核心 Trade-off

1. **低 Dk/Df vs 高 Peel**：需要增韌劑補償
2. **低 CTE vs 高 Peel**：填料量需要精確控制在 40% 左右
3. **高 Peel vs 高 Tg**：增韌劑量需要限制在 3% 以下

### 下一步

1. 用真實實驗數據驗證模型
2. 調整模型參數以符合實際情況
3. 建立互動式 Dashboard 供配方工程師使用